In [ ]:
!pip install transformers datasets torch pandas scikit-learn matplotlib seaborn


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 55.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 18.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 29.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 15.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 86.0 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitli

In [ ]:
# Step 2: Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datasets import load_dataset, Dataset
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer,
    pipeline
)
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import torch
from torch.utils.data import DataLoader
import warnings
warnings.filterwarnings('ignore')

print("✅ All libraries imported successfully!")

✅ All libraries imported successfully!


In [ ]:
# Step 3: Load and explore the dataset
print("📊 Loading dataset...")
dataset = load_dataset("sharjeelyunus/github-issues-dataset")

# Convert to pandas for easier exploration
df = dataset['train'].to_pandas()
print(f"Dataset shape: {df.shape}")
print(f"Columns: {df.columns.tolist()}")
print("\n📋 First few rows:")
print(df.head())

📊 Loading dataset...


README.md: 0.00B [00:00, ?B/s]

github_issues_dataset.parquet:   0%|          | 0.00/139M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/114073 [00:00<?, ? examples/s]

Dataset shape: (114073, 7)
Columns: ['id', 'repo', 'title', 'body', 'labels', 'priority', 'severity']

📋 First few rows:
        id        repo                                              title  \
0   393061  youtube-dl                     Output file size with -s or -g   
1  1637737  youtube-dl                     Create a php API and demo page   
2  1639054  youtube-dl  integrate template "special sequences" in help...   
3  1789251  youtube-dl                  Add a path option to --keep-video   
4  1789512  youtube-dl   add support for picasaweb.google.com video clips   

                                                body                labels  \
0  Was: http://bitbucket.org/rg3/youtube-dl/issue...               request   
1  youtube-dl is often embedded by php applicatio...                   php   
2  like in http://rg3.github.com/youtube-dl/docum...               request   
3  Hey there,\n\nI think it would be a great idea...               request   
4  > /opt/local/bin/youtub

In [ ]:
# Step 4: Data exploration and preprocessing
print("\n🔍 Dataset Info:")
print(df.info())
print("\n📈 Target distribution:")
if 'severity' in df.columns:
    print(df['severity'].value_counts())
elif 'label' in df.columns:
    print(df['label'].value_counts())
else:
    print("Available columns:", df.columns.tolist())
    print("Please check which column contains severity labels")

# Check for missing values
print(f"\n❌ Missing values:\n{df.isnull().sum()}")


🔍 Dataset Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 114073 entries, 0 to 114072
Data columns (total 7 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   id        114073 non-null  int64 
 1   repo      114073 non-null  object
 2   title     114073 non-null  object
 3   body      113940 non-null  object
 4   labels    114073 non-null  object
 5   priority  114073 non-null  object
 6   severity  114073 non-null  object
dtypes: int64(1), object(6)
memory usage: 6.1+ MB
None

📈 Target distribution:
severity
Critical    66491
Minor       29769
Major       17813
Name: count, dtype: int64

❌ Missing values:
id            0
repo          0
title         0
body        133
labels        0
priority      0
severity      0
dtype: int64


In [ ]:
# Step 5: Prepare data for BERT
# Combine title and body for better context (adjust column names as needed)
def prepare_text_data(df):
    """Prepare text data by combining relevant fields"""
    text_columns = []

    # Check available text columns
    if 'title' in df.columns:
        text_columns.append('title')
    if 'body' in df.columns:
        text_columns.append('body')
    if 'description' in df.columns:
        text_columns.append('description')

    if len(text_columns) == 0:
        # If no standard columns, use the first text-like column
        for col in df.columns:
            if df[col].dtype == 'object':
                text_columns.append(col)
                break

    # Combine text fields
    if len(text_columns) > 1:
        df['combined_text'] = df[text_columns].fillna('').apply(
            lambda x: ' '.join(x.astype(str)), axis=1
        )
    else:
        df['combined_text'] = df[text_columns[0]].fillna('')

    return df

df = prepare_text_data(df)

In [ ]:
# Step 6: Define severity mapping (adjust based on your dataset)
def map_severity_labels(df):
    """Map severity labels to numerical values"""
    severity_col = None

    # Find severity column
    for col in ['severity', 'label', 'priority', 'level']:
        if col in df.columns:
            severity_col = col
            break

    if severity_col is None:
        print("⚠️ No severity column found. Creating dummy labels for demo.")
        df['severity'] = np.random.choice(['low', 'medium', 'high'], size=len(df))
        severity_col = 'severity'

    # Create label mapping
    unique_labels = df[severity_col].unique()
    label_to_id = {label: idx for idx, label in enumerate(unique_labels)}
    id_to_label = {idx: label for label, idx in label_to_id.items()}

    df['labels'] = df[severity_col].map(label_to_id)

    print(f"📝 Label mapping: {label_to_id}")
    return df, label_to_id, id_to_label

df, label_to_id, id_to_label = map_severity_labels(df)

📝 Label mapping: {'Critical': 0, 'Major': 1, 'Minor': 2}


In [ ]:
# Step 7: Split the data
print("✂️ Splitting data...")
train_df, test_df = train_test_split(
    df,
    test_size=0.2,
    random_state=42,
    stratify=df['labels']
)

train_df, val_df = train_test_split(
    train_df,
    test_size=0.2,
    random_state=42,
    stratify=train_df['labels']
)

print(f"Training set: {len(train_df)} samples")
print(f"Validation set: {len(val_df)} samples")
print(f"Test set: {len(test_df)} samples")

✂️ Splitting data...
Training set: 73006 samples
Validation set: 18252 samples
Test set: 22815 samples


In [ ]:
# Step 8: Initialize BERT tokenizer and model
print("🤖 Loading BERT model...")
model_name = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=len(label_to_id)
)

🤖 Loading BERT model...


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# Step 9: Memory-efficient tokenization
def tokenize_data_batch(texts, tokenizer, max_length=256, batch_size=1000):
    """Memory-efficient tokenization with batching"""
    print(f"🔤 Tokenizing {len(texts)} texts in batches of {batch_size}...")

    all_input_ids = []
    all_attention_masks = []

    # Process in smaller batches
    for i in range(0, len(texts), batch_size):
        batch_texts = texts[i:i + batch_size].tolist()
        print(f"  Processing batch {i//batch_size + 1}/{(len(texts)-1)//batch_size + 1}")

        # Tokenize batch
        batch_encoding = tokenizer(
            batch_texts,
            truncation=True,
            padding=True,
            max_length=max_length,
            return_tensors="pt"
        )

        # Store results
        all_input_ids.append(batch_encoding['input_ids'])
        all_attention_masks.append(batch_encoding['attention_mask'])

        # Clear memory
        del batch_encoding
        torch.cuda.empty_cache() if torch.cuda.is_available() else None

    # Combine all batches
    final_input_ids = torch.cat(all_input_ids, dim=0)
    final_attention_masks = torch.cat(all_attention_masks, dim=0)

    return {
        'input_ids': final_input_ids,
        'attention_mask': final_attention_masks
    }

# SOLUTION 1: Reduce dataset size for testing
print("🔍 Checking dataset size...")
if len(df) > 50000:
    print(f"⚠️ Large dataset detected ({len(df)} samples). Using subset for training.")
    df_subset = df.sample(n=50000, random_state=42)  # Use 100k samples
    train_df, test_df = train_test_split(df_subset, test_size=0.2, random_state=42, stratify=df_subset['labels'])
    train_df, val_df = train_test_split(train_df, test_size=0.2, random_state=42, stratify=train_df['labels'])
    print(f"✅ Using subset - Training: {len(train_df)}, Val: {len(val_df)}, Test: {len(test_df)}")

# SOLUTION 2: Clean up memory before tokenization
import gc
gc.collect()
torch.cuda.empty_cache() if torch.cuda.is_available() else None

print("🔤 Starting memory-efficient tokenization...")
# Reduced max_length to save memory (256 instead of 512)
train_encodings = tokenize_data_batch(train_df['combined_text'], tokenizer, max_length=256, batch_size=500)
print("✅ Training data tokenized")

val_encodings = tokenize_data_batch(val_df['combined_text'], tokenizer, max_length=256, batch_size=500)
print("✅ Validation data tokenized")

test_encodings = tokenize_data_batch(test_df['combined_text'], tokenizer, max_length=256, batch_size=500)
print("✅ Test data tokenized")

# Clear memory after tokenization
gc.collect()
torch.cuda.empty_cache() if torch.cuda.is_available() else None

🔍 Checking dataset size...
⚠️ Large dataset detected (114073 samples). Using subset for training.
✅ Using subset - Training: 32000, Val: 8000, Test: 10000
🔤 Starting memory-efficient tokenization...
🔤 Tokenizing 32000 texts in batches of 500...
  Processing batch 1/64
  Processing batch 2/64
  Processing batch 3/64
  Processing batch 4/64
  Processing batch 5/64
  Processing batch 6/64
  Processing batch 7/64
  Processing batch 8/64
  Processing batch 9/64
  Processing batch 10/64
  Processing batch 11/64
  Processing batch 12/64
  Processing batch 13/64
  Processing batch 14/64
  Processing batch 15/64
  Processing batch 16/64
  Processing batch 17/64
  Processing batch 18/64
  Processing batch 19/64
  Processing batch 20/64
  Processing batch 21/64
  Processing batch 22/64
  Processing batch 23/64
  Processing batch 24/64
  Processing batch 25/64
  Processing batch 26/64
  Processing batch 27/64
  Processing batch 28/64
  Processing batch 29/64
  Processing batch 30/64
  Processing b

In [ ]:
# Step 10: Create Dataset class
class IssueDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

# Create datasets
train_dataset = IssueDataset(train_encodings, train_df['labels'].tolist())
val_dataset = IssueDataset(val_encodings, val_df['labels'].tolist())
test_dataset = IssueDataset(test_encodings, test_df['labels'].tolist())

In [ ]:
# Step 11: Safe training arguments (multiple error fixes)
import os

# Create output directory if it doesn't exist
os.makedirs('./results', exist_ok=True)
os.makedirs('./logs', exist_ok=True)

# SAFE training arguments with error handling
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=2,
    per_device_train_batch_size=2,  # Further reduced to prevent memory issues
    per_device_eval_batch_size=4,
    warmup_steps=100,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=50,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    greater_is_better=False,
    # Removed potentially problematic arguments
    gradient_accumulation_steps=4,  # Increased to compensate for smaller batch
    remove_unused_columns=False,  # Prevent column removal issues
    report_to=None,  # Disable wandb/tensorboard to avoid errors
)
print("✅ Training arguments created successfully!")


# Verify datasets exist before proceeding
print(f"📊 Dataset sizes:")
print(f"  Training: {len(train_dataset)} samples")
print(f"  Validation: {len(val_dataset)} samples")
print(f"  Test: {len(test_dataset)} samples")

✅ Training arguments created successfully!
📊 Dataset sizes:
  Training: 32000 samples
  Validation: 8000 samples
  Test: 10000 samples


In [ ]:
# Step 12: Define metrics
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return {
        'accuracy': accuracy_score(labels, predictions)
    }

In [ ]:
# Step 13: Initialize trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
)

In [ ]:
# Step 14: Train the model
print("🚀 Starting training...")
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


🚀 Starting training...


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: sahumayank689 (sahumayank689-iacsd) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Epoch,Training Loss,Validation Loss


Epoch,Training Loss,Validation Loss


In [ ]:

# Step 15: Evaluate on test set
print("📊 Evaluating on test set...")
test_results = trainer.evaluate(test_dataset)
print(f"Test Results: {test_results}")

In [ ]:
# Step 16: Make predictions
predictions = trainer.predict(test_dataset)
y_pred = np.argmax(predictions.predictions, axis=1)
y_true = test_df['labels'].tolist()

In [ ]:
# Step 17: Generate detailed results
print("📈 Classification Report:")
print(classification_report(y_true, y_pred, target_names=list(label_to_id.keys())))

In [ ]:

# Confusion Matrix
plt.figure(figsize=(8, 6))
cm = confusion_matrix(y_true, y_pred)
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=list(label_to_id.keys()),
            yticklabels=list(label_to_id.keys()))
plt.title('Confusion Matrix')
plt.ylabel('True Label')
plt.xlabel('Predicted Label')
plt.show()

In [ ]:
# Step 18: Save the model
print("💾 Saving model...")
model.save_pretrained('./issue-severity-classifier')
tokenizer.save_pretrained('./issue-severity-classifier')

In [ ]:
# Step 19: Create inference pipeline
print("🎯 Creating inference pipeline...")
classifier = pipeline(
    "text-classification",
    model=model,
    tokenizer=tokenizer,
    return_all_scores=True
)

In [ ]:
# Step 20: Test with sample predictions
def predict_severity(text):
    """Predict severity for a given text"""
    results = classifier(text)
    predictions = {id_to_label[i]: score['score'] for i, score in enumerate(results[0])}
    predicted_label = max(predictions, key=predictions.get)
    confidence = max(predictions.values())

    return predicted_label, confidence, predictions

In [ ]:
# Example predictions
sample_issues = [
    "Critical bug: Application crashes on startup",
    "Enhancement: Add dark mode theme",
    "Minor typo in documentation"
]

print("🔮 Sample Predictions:")
for issue in sample_issues:
    pred_label, confidence, all_scores = predict_severity(issue)
    print(f"\nIssue: '{issue}'")
    print(f"Predicted Severity: {pred_label} (Confidence: {confidence:.3f})")
    print(f"All scores: {all_scores}")

In [ ]:
# Step 21: Model performance summary
print("\n" + "="*50)
print("🎉 MODEL TRAINING COMPLETE!")
print("="*50)
print(f"✅ Final Test Accuracy: {accuracy_score(y_true, y_pred):.3f}")
print(f"📁 Model saved to: './issue-severity-classifier'")
print(f"🏷️ Number of classes: {len(label_to_id)}")
print(f"📊 Training samples: {len(train_df)}")
print(f"🔍 Test samples: {len(test_df)}")

In [ ]:
# Step 22: Test with new, unseen data
new_issues = [
    "	Output file size with -s or -g",
    "The login button is not working on the mobile app.",
    "integrate template special sequences in help output",
    "Download range",
    "The API is returning a 500 error for all requests.",
    "The user interface is slow and unresponsive at times.",
    "The color of the main header is not consistent with the brand guidelines."
]

print("🔮 Predicting severity for new issues:")
for issue in new_issues:
    pred_label, confidence, all_scores = predict_severity(issue)
    print(f"\nIssue: '{issue}'")
    print(f"Predicted Severity: {pred_label} (Confidence: {confidence:.3f})")
    print(f"All scores: {all_scores}")